---
title: "."
title-slide-attributes:
    data-background-image: "logo.PNG"
    data-background-size: cover
format:
  revealjs:
    theme: simple  # 기본 테마(원하는 테마 변경 가능)
    css: styles.css
    slide-number: true  # 페이지 번호 표시
    footer: "Team 6 | LS BigData School | 2025"  # 푸터 설정
    logo: "logo3.png"  # 로고 삽입
    echo: true
---


## 1. 분석 개요
- 항공편 지연은 항공사, 공항 운영, 승객에게 큰 영향을 미치는 중요한 요소
- 지연을 유발하는 다양한 요인(탑승률, 항공기 모델, 노선 등)에 대한 분석 필요

#### 분석 목표:
- 탑승률과 지연의 관계 분석
- 기체 제조사/모델별 지연 시간 분석
- 지연 원인에 대한 세부 원인 분석


---

## 2. 사용 데이터

| 데이터셋       | 데이터 출처     | 설명                                       |
|---------------|-----------------|-------------------------------------------|
| `flights.csv`  | 내부 데이터 | 항공편 정보 (출발지, 도착지, 지연 시간 등) |
| `planes.csv`   | 내부 데이터 | 항공기 정보 (제조 연도, 모델 등)           |
| `airlines.csv` | 내부 데이터 | 항공사 코드 및 항공사명 정보               |
| `t100.csv`     | 외부 데이터 | 미국 국내선 항공사의 운항 실적 (항공편 수, 좌석 수, 승객 수, 운항 거리 등) 제공 |


In [ ]:
#| echo: false
from nycflights13 import flights, planes, airlines
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

plane_df = planes.rename(columns={'year': 'manuf_year'})
df = pd.merge(flights, plane_df, on='tailnum')
pass_df = pd.read_csv('../ppt/data/t100.csv')

---

## 3. 공통 데이터 전처리


1. `flights.csv`와 `planes.csv`를 `tailnum` 기준으로 병합
2. `datetime` 컬럼 생성 (`year`, `month`, `day`, `hour`, `minute`를 변환)
3. `airlines.csv`를 `carrier` 기준으로 병합하여 항공사명 추가


In [ ]:
df = pd.merge(flights, plane_df, on='tailnum')
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute']])
df = df.merge(airlines, on="carrier", how="left")

In [ ]:
#|echo: false
pass_df.columns = pass_df.columns.str.lower()

---

## 4. 분석

<div id="subject_list">
#### 4.1 탑승률과 항공편 지연 관계 분석
#### 4.2 제조사별 엔진 지연 시간 분석
#### 4.3 제조사별 엔진 지연 시간 원인 도출
</div>

---

### 4.1 탑승률과 항공편 지연 관계 분석

- **만석에 가까울수록** 탑승 절차, 수하물 처리 등이 길어져 **지연**될 가능성이 있음

<div id='font-red'>
 -> 탑승률이 높을수록 항공편 출발 지연이 증가하는가?
</div>

---

<div class="section-title">4.1 탑승률과 항공편 지연 관계</div>

#### 추가 데이터 전처리

- `t100.csv`에서 `origin`, `dest`, `carrier`가 기존 `flights.csv`에 존재하는 항목들만 남김


In [ ]:
#| echo: true
# origin
pass_df = pass_df[pass_df['origin'].isin(['EWR', 'LGA', 'JFK'])]

# dest
dest_list = df['dest'].unique()
pass_df = pass_df[pass_df['dest'].isin(dest_list)]

# carrier
carrier_list = df['carrier'].unique()
pass_df = pass_df[pass_df['carrier'].isin(carrier_list)]

- `year`, `month`, `carrier`, `origin`, `dest` 기준으로 `seats`, `passenger` 합산


In [ ]:
pass_grouped = pass_df.groupby(['year', 'month', 'carrier', 'origin', 'dest'])[['seats', 'passengers']].sum().reset_index()

---

- `flights.csv`를 년/월/항공사/출발지/목적지 를 기준으로 그룹화하여 `flight_count`, `avg_dep_delay` 등을 계산


In [ ]:
#| code-fold: true
flight_grouped = df.groupby(['year', 'month', 'carrier', 'origin', 'dest']).agg(
    flight_count=('flight', 'count'),
    avg_dep_delay=('dep_delay', 'mean'),
    avg_arr_delay=('arr_delay', 'mean'),
    total_distance=('distance', 'sum')
    ).reset_index()

<br>

- 최종적으로 두 데이터를 병합 후, 탑승률(`passengers`, `seats`) 계산 


In [ ]:
#| code-fold: true
merged_df = flight_grouped.merge(
    pass_grouped,
    on=['year', 'month', 'carrier', 'origin', 'dest'],
)

merged_df['load_factor'] = merged_df['passengers'] / merged_df['seats']

<br>

- 이상치 제거(IQR 방식)


In [ ]:
#| echo: true
#| code-fold: true
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)  # 1사분위수 (Q1)
    Q3 = df[column].quantile(0.75)  # 3사분위수 (Q3)
    IQR = Q3 - Q1  # IQR 계산
    lower_bound = Q1 - 1.5 * IQR  # 이상치 하한
    upper_bound = Q3 + 1.5 * IQR  # 이상치 상한
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

In [ ]:
#| code-fold: true
cleaned_df = remove_outliers(merged_df, "avg_dep_delay")

---


#### 분석 및 시각화 


In [ ]:
#| code-fold: true
cleaned_df["load_factor_bin"] = pd.cut(cleaned_df["load_factor"], bins=[0, 0.3, 0.5, 0.7, 0.9, 1], labels=["0-30%", "30-50%", "50-70%", "70-90%", "90-100%"])
summary_stats = cleaned_df.groupby("load_factor_bin")[["avg_dep_delay", "avg_arr_delay"]].mean().reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(data=summary_stats, x="load_factor_bin", y="avg_dep_delay", color="blue")

plt.xlabel("Load Factor Range")
plt.ylabel("Average Departure Delay (minutes)")
plt.title("Average Departure Delay by Load Factor")
plt.grid(axis="y")
plt.show()

---

#### 결과

---

### 4.2 제조사별 엔진 Delay 시간 분석

- 최근 화두가 되는 비행기 엔진문제
- 그 중 '보잉'사의 엔진 문제가 많은 이야깃거리 
- 항공기 제조사중 가장 출발 지연이 가장 많은 제조사와 그 기종은?

---

#### 제조사들의 결함 문제

![](img\EMB.png){style="position: absolute; width: 80%; height: 90%;"}
![](img\BOE.png){style="position: absolute; bottom: 50px; right: 50px; width: 50%; height: 60%;"}

---

<div class="section-title">4.1 탑승률과 항공편 지연 관계</div>

#### 추가 데이터 전처리

- 도착 지연시간과 출발 지연시간을 더하여 새로운 column 생성

In [ ]:
df['delay_time'] = df['arr_delay'] + df['dep_delay']
print(df['delay_time'].head())

---


- 제조사별 모델을 그룹화하여 delay 평균과 비행 횟수를 구함

In [ ]:
df_air_time = (
    df.groupby(['manufacturer', 'model'])["delay_time"]
    .agg(['mean', 'count']).sort_values('mean', ascending=False)  
).reset_index()
df_air_time.head()

---

- 운항 횟수가 1000회 미만인 데이터를 제거하여 신뢰성 있는 데이터만 유지 (147개 -> 38개)
- 출발 지연시간이 가장 긴 상위 10개 및 가장 짧은 하위 10개 기종 추출
- .nlargest(), nsmallest() 를 사용해서 상,하위 10개의 'mean'열 데이터 추출


In [ ]:
#| code-fold: true
df_air_time = df_air_time.drop(df_air_time[df_air_time['count']<1000].index)
df_air_time.shape   # 기존 147개 데이터에서 38개의 데이터로 축소

delay_time_top = df_air_time.nlargest(10, 'mean')
delay_time_bottom = df_air_time.nsmallest(10, 'mean')

::: {.column width="50%"}

상위 10개 추출

In [ ]:
#| echo: false
print(delay_time_top[['model','mean','count']])

:::

:::{.column width="50%"}

하위 10개 추출

In [ ]:
#| echo: false
print(delay_time_bottom[['model','mean','count']])

:::

---

- 그래프에서 제조사와 모델을 직관적으로 표시하기 위해 새로운 열 manu_model 생성
- 제조사 이름의 앞 3글자 + 두 번째 단어의 첫 글자 + 모델명을 조합하여 가독성을 높임


In [ ]:
#|code-fold: true
delay_time_top['manu_model'] = ( delay_time_top['manufacturer'].str[:3] +  delay_time_top['manufacturer'].str.split().str[1].str[0].fillna(''))+ "/" + delay_time_top['model']

delay_time_bottom['manu_model'] = ( delay_time_bottom['manufacturer'].str[:3] +  delay_time_bottom['manufacturer'].str.split().str[1].str[0].fillna(''))+ "/" + delay_time_bottom['model']


In [ ]:
#| echo: false
print(delay_time_top[['manufacturer','model','manu_model']])

---

#### 분석 및 시각화 


In [ ]:
#| code-fold: true
plt.figure(figsize=(14,5))
sns.barplot(x=delay_time_top['manu_model'],y=delay_time_top['mean'], palette="Reds_r")
plt.ylim(0, 50)
plt.xlabel("model by manufacturer",fontsize=20)
plt.ylabel("delay time_mean",fontsize=20)
plt.title("Bad flights",fontsize=20)
plt.xticks(rotation=40,fontsize=16, fontweight='bold');

---

#### 분석 및 시각화 


In [ ]:
#| code-fold: true
plt.figure(figsize=(14,5))
sns.barplot(x=delay_time_bottom['manu_model'],y=delay_time_bottom['mean'], palette="Blues_r")
plt.ylim(-5, 50)
plt.xlabel("model by manufacturer",fontsize=20)
plt.ylabel("delay time_mean",fontsize=20)
plt.title("Good flights",fontsize=20)
plt.xticks(rotation=40,fontsize=16, fontweight='bold');

---

#### 결론

- 가장 신뢰성이 떨어지는 기종 Top3는 '**EMB**'사의 '**EMB-145LR**'기종, '**BOE**'사의 '**717-200**'기종 , '**CAN**'사의 '**CL-600-2B19**'기종이다
- 또한 제조사만 봤을때 '**BOMI**'사 **4대**, '**EMB**'사 **2대** , '**BOE**'사 **2대**가 나타나여 이 제조사들의 model이 지연이 잘되는 걸 알 수 있다.
- EMB는 엠브라에르, BOE는 보잉, CAN은 케나다에어


<br>


- 반대로 신뢰성이 높은 기종 Top3는 '**BOE**'사의 '**757-231**'기종, '**BOE**'사의 '**757-2Q8**' 기종, '**AIR**'사의 '**A321-211**'기종 이다.
- 또한 제조사만 봤을때 '**BOE**'사 **5대**, '**AIRBUS**'사 **2대**, '**AIR Industry**'사 **2대**로 신뢰성 높은  model을 만들어 내는것을 알 수 있다.

---

### 피해야 할 비행기


<div class="plane_images">

<div class="plane_image">
<h3 style="font-size: 30px;">ERJ 145</h3>
![](img\erj145_kp.jpg)
</div>


<div class="plane_image">
<h3 style="font-size: 30px;">BOE 717</h3>
![](img\boe717.jpg)
</div>


<div class="plane_image">
<h3 style="font-size: 30px;">CL-600-2B19</h3>
![](img\CL-600-2B19.jpg)
</div>

</div>

---

#### 4.3 제조사별 엔진 지연 시간 원인 도출

- 앞선 결과에서 더 자세한 원인을 분석하고자함


In [ ]:
#| code-fold: true
delay_time_top["flight_type"] = "bad flight"
delay_time_bottom["flight_type"] = "good flight"
concat_data = pd.concat([delay_time_top, delay_time_bottom], axis=0)
df = pd.merge(concat_data,df, on=['manufacturer','model'])

delay_time_bottom.head(2)
concat_data.columns
df.columns

---


In [ ]:
#| code-fold: true
count_avg1 = df.groupby(['manufacturer','manu_model','flight_type','manuf_year'])['delay_time'].agg(['mean','count']).reset_index()

count_avg1.head(2)

---


In [ ]:
#| code-fold: true
sns.scatterplot(data=count_avg1, x='manuf_year', y='mean', hue='flight_type', palette='viridis')
plt.legend(title='flight_type', bbox_to_anchor=(1.05, 0.5), loc='center left')

# 그래프 제목과 레이블
plt.title('Scatterplot: year vs Departure Delay')
plt.xlabel('Year')
plt.ylabel('Average Delay')

# 그래프 출력
plt.show()

---


In [ ]:
#| code-fold: true
sns.scatterplot(data=count_avg1, x='manuf_year', y='mean', hue='manufacturer', style='flight_type', palette='tab10',s=100)

plt.legend(title='Manufacturer', bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.legend(loc='upper left', fontsize=5)
# 그래프 제목과 레이블
plt.title('Scatterplot: year vs Departure Delay')
plt.xlabel('Year')
plt.ylabel('Average Delay')
plt.show()